In [101]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Parameters
sequence_length = 20
num_samples = 300  
target_size = (64, 64)
batch_size = 1
num_classes = 4

In [102]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]
)

train_generator = datagen.flow_from_directory(
    'Data/AugmentedAlzheimerDataset',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    'Data/AugmentedAlzheimerDataset',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 27188 images belonging to 4 classes.
Found 6796 images belonging to 4 classes.


In [103]:
train_generator = datagen.flow_from_directory(
    'Data/AugmentedAlzheimerDataset',
    target_size=(64, 64),
    batch_size=1,
    class_mode='categorical',
    subset='training'  
)

validation_generator = datagen.flow_from_directory(
    'Data/AugmentedAlzheimerDataset',
    target_size=(64, 64),
    batch_size=1,
    class_mode='categorical',
    subset='validation'  
)

Found 27188 images belonging to 4 classes.
Found 6796 images belonging to 4 classes.


In [104]:
base_cnn = VGG16(weights='imagenet', include_top=False, input_shape=(*target_size, 3))
cnn_model = Sequential([
    base_cnn,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.3)
])

In [105]:
train_sequences = np.zeros((num_samples, sequence_length, 256))
train_labels = []
for i in range(num_samples):
    sequence_labels = []
    for j in range(sequence_length):
        img, label = next(train_generator)
        feature_vector = cnn_model.predict(img)
        train_sequences[i, j, :] = feature_vector.flatten()
    train_labels.append(label[0])

train_labels = np.array(train_labels)

val_sequences = np.zeros((num_samples // 5, sequence_length, 256))  
val_labels = []
for i in range(num_samples // 5):
    sequence_labels = []
    for j in range(sequence_length):
        img, label = next(val_generator)
        feature_vector = cnn_model.predict(img)
        val_sequences[i, j, :] = feature_vector.flatten()
    val_labels.append(label[0])

val_labels = np.array(val_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━

In [106]:

model = Sequential([
    LSTM(100, activation='relu', return_sequences=True, input_shape=(sequence_length, 256)),
    Dropout(0.3),
    LSTM(50, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

In [107]:
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


In [108]:
history = model.fit(
    train_sequences, train_labels,
    epochs=20,
    batch_size=16,
    validation_data=(val_sequences, val_labels),
    callbacks=[lr_scheduler]
)

Epoch 1/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.2085 - loss: 1.4544 - val_accuracy: 0.3000 - val_loss: 1.3895 - learning_rate: 1.0000e-04
Epoch 2/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2750 - loss: 1.3956 - val_accuracy: 0.2667 - val_loss: 1.3676 - learning_rate: 1.0000e-04
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2754 - loss: 1.3650 - val_accuracy: 0.3167 - val_loss: 1.3645 - learning_rate: 1.0000e-04
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3103 - loss: 1.3649 - val_accuracy: 0.2333 - val_loss: 1.3564 - learning_rate: 1.0000e-04
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2770 - loss: 1.3725 - val_accuracy: 0.3167 - val_loss: 1.3649 - learning_rate: 1.0000e-04
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2733 - loss: 1.3775 - val_accuracy: 0.3167 - val_loss: 1.3640 - learning_rate: 1.0000e-04
Epoch 7/20
15/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2883 - los

In [109]:
val_loss, val_accuracy = model.evaluate(val_sequences, val_labels)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.2f}")

model.save("lstm_bci_model.keras")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2944 - loss: 1.3677 
Validation Loss: 1.3646
Validation Accuracy: 0.32
